In [1]:
import torch 
import numpy as np 
import os
import re 
import random 

from torch.utils.data import Dataset 

In [2]:
filename = '../data/maptask/splits/train.txt'
dialog_files = os.listdir('../data/maptask/parsed_transcripts')

In [3]:
FILEDIR = '../data/maptask/transcripts'

In [4]:
# Parsing transcript 
for file in os.listdir(FILEDIR)[:5]:
    print("CONVO")
    speaker = "" 
    output  = ""
    with open(os.path.join(FILEDIR, file)) as stream:
        lines = stream.readlines()[4:]
        for line in lines:
            output += line
        print(output)


CONVO
f	uh-huh 	
g	right 	
g	go along to your left in a straight line about an inch on the map 	
f	what am i trying to avoid first of all 	
g	stony desert 	
f	that's below the start 	
g	uh-huh 	
f	so i just go s-- left 	
g	go along left 	
f	how far 	
g	about centimetre and a half 	
f	aye 	
g	you past it 	
f	oh right okay right okay 	
g	you past stony desert 	
f	yeah 	
g	right now go down to bottom left t-- 	
f	diagonally 	
g	yes 	
g	to miss the to get below the plane crash 	
f	the plane crash at the top right 	
g	right 	
f	where does your route end 	
g	sort of middle above middle right 	
f	what's down there 	
g	oh there's another plane crash down here though 	
f	no there's only one 	
g	right we'll just do that anyway right go down diagonally to the bottom left of the page 	
f	right 	
g	not right to the bottom 	
f	right okay 	
g	and then up in a straight line about 	
f	right 	
g	er about two and a half centimetres 	
f	is there a collaps-- 	
f	is there a collapsed shelter 	
g	collapsed s

In [27]:
# Parsing transcript 
for file in os.listdir(FILEDIR)[:5]:
    print("CONVO")
    speaker = "" 
    output  = ""
    with open(os.path.join(FILEDIR, file)) as stream:
        lines = stream.readlines()[4:]
        for line in lines:
            line = line.replace("\n", "")
            new_speaker = line[0]

            if new_speaker == speaker or speaker == "":
                output += line[2:]
                if speaker == "":
                    speaker = new_speaker
            else:
                output += "\n"
                output += line[2:]
                speaker = new_speaker
        output = re.sub('[^\S\r\n]+', ' ', output)
        print(output)


CONVO
uh-huh 
right go along to your left in a straight line about an inch on the map 
what am i trying to avoid first of all 
stony desert 
that's below the start 
uh-huh 
so i just go s-- left 
go along left 
how far 
about centimetre and a half 
aye 
you past it 
oh right okay right okay 
you past stony desert 
yeah 
right now go down to bottom left t-- 
diagonally 
yes to miss the to get below the plane crash 
the plane crash at the top right 
right 
where does your route end 
sort of middle above middle right 
what's down there 
oh there's another plane crash down here though 
no there's only one 
right we'll just do that anyway right go down diagonally to the bottom left of the page 
right 
not right to the bottom 
right okay 
and then up in a straight line about 
right 
er about two and a half centimetres 
is there a collaps-- is there a collapsed shelter 
collapsed shelter go up round above that 
uh-huh right so i start swinging to the right again 
well once you've gone up 'til

In [32]:
dialog = []
labels = []
with open(filename, "r") as indexes:
    for index in indexes:
        dialog_file = os.path.join('../data/maptask/parsed_transcripts', dialog_files[int(index)])
        
        with open(dialog_file, "r") as d:
            for line in d.readlines():
                    dialog.append(line.strip())
                    labels.append(1)

In [33]:
def weighted_random(choices, first=1, last=1):
    choices = [0] * first + [x for x in range(1, len(choices) - 1)] + [len(choices) - 1] * last 
    return random.choices(choices)

In [36]:
def add_non_trp(ipu):
    words =  ipu.split(" ")
    start_index = weighted_random(range(0,len(words) - 2), first=10)[0]
    length = weighted_random(range(1, len(words) - start_index), last=9)[0]
    return " ".join(words[start_index: start_index+length])

In [37]:
for x in dialog:
    print("STP")
    print(x)
    print(add_non_trp(x))

STP
right er the start is at a diamond mine
right er the start is at a
STP
yes

STP
now we're do this the same way as we did it before there's a wee man walking along
we're do this the same way as we did it before there's a wee man
STP
uh-huh

STP
so as we can er it'll be fae the perspective of the viewer right is is the guy walking along
can er it'll be fae the perspective of the viewer right is is the
STP
how can i right
how can
STP
right the guy is standing so that the diamond mine will be on his left-hand side
right the guy
STP
okay

STP
mmhmm

STP
right

STP
now you walk down just past it
you walk down
STP
just past the diamond mine
just past the
STP
aye and turn to your left
aye and turn to
STP
so that's in below the diamond mine towards the centre of the page
so that's in below the diamond mine towards
STP
and that's correct
and
STP
okay

STP
right now there's a graveyard on your left-hand side
right now there's a graveyard
STP
no the diamond mine's on my left-hand side
no the d

In [41]:
po = 10
itera = 0
for i in range(po):
    itera += 1
    if i % 2 == 0:
        po += 1
print("fndjisn", po, itera)

fndjisn 15 10
